In [44]:
# Import Statements:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

# Creating Training and Testing Data:

transform = transforms.ToTensor()

train_data = []
test_data = []

folder_path = 'MedicalMNIST'

classifier = 0

for sub_folder in os.listdir(folder_path):
    
    if sub_folder != '.DS_Store':

        ix = 0
        print(f'Classifier {sub_folder} = {classifier}')
        
        for image_path in os.listdir(f'{folder_path}/{sub_folder}'):

            if image_path != '.DS_Store':
            
                image = Image.open(f'{folder_path}/{sub_folder}/{image_path}')
                image = transform(image)
                
                if ix < 8000:
                    train_data.append([image, classifier])
                else:
                    test_data.append([image, classifier])

                ix += 1
                    
                
        classifier += 1


Classifier Hand = 0
Classifier BreastMRI = 1
Classifier ChestCT = 2
Classifier HeadCT = 3
Classifier AbdomenCT = 4
Classifier CXR = 5


In [49]:
# Batching Training and Testing Data:

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Creating Convolutional Neural Network:

class ConvolutionalNeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()

        # Creating Convolutional Layers:

        self.conv1 = nn.Conv2d(1, 8, 3, 1)
        self.conv2 = nn.Conv2d(8, 24, 3, 1)

        # Creating Fully-Connected Layers:

        self.fc1 = nn.Linear(24*14*14, 100)
        self.out = nn.Linear(100, 6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 24*14*14)

        x = F.relu(self.fc1(x))
        x = self.out(x)

        return x

model = ConvolutionalNeuralNetwork()

# Creating Constants:

LR = 3e-4
EPOCHS = 10

# Creating Criterion and Optimizer:

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Creating Training Loop:

for epoch in range(EPOCHS):

    for batch, (x, y) in enumerate(train_loader):

        batch += 1

        y_pred = model(x)

        loss = criterion(y_pred, y)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    print(f'Epoch: {epoch} Loss: {loss:.4f}')



Epoch: 0 Loss: 0.0074
Epoch: 1 Loss: 0.0014
Epoch: 2 Loss: 0.0005
Epoch: 3 Loss: 0.0000
Epoch: 4 Loss: 0.0015
Epoch: 5 Loss: 0.0001
Epoch: 6 Loss: 0.0000
Epoch: 7 Loss: 0.1146
Epoch: 8 Loss: 0.0000
Epoch: 9 Loss: 0.0000


In [61]:
# Evaluating Model Accuracy:

correct = 0
ix = 0

for (x, y) in test_loader:

    for batch in range(32):

        ix += 1

        with torch.inference_mode():

            prediction = torch.argmax(model(x[batch])).item()

            actual = y[batch].item()

            if prediction == actual:
                correct += 1

            if (ix == 10000):
                break

    if (ix == 10000):
        break
        

print(f'The Model Made {correct} / {ix} Predictions...')   

The Model Made 9995 / 10000 Predictions...
